In [ ]:
import os
os.chdir("..")

In [ ]:
os.getcwd()

In [ ]:
from datasets import load_from_disk

In [ ]:
data = load_from_disk("artifacts/data/ingestion/ingested/samsum_dataset/train")

In [ ]:
data

In [ ]:
data.num_rows

In [ ]:
data.features

In [ ]:
data[0]

In [ ]:
data[:3]

In [ ]:
data[0]["id"]

In [ ]:
data[0]["dialogue"]

In [ ]:
data[0]["summary"]

In [ ]:
print(type(data["dialogue"]))
data["dialogue"]

In [ ]:
print(type(data["dialogue"][0]))
data["dialogue"][0]

In [ ]:
print(type(data["summary"][0]))
data["summary"][0]

In [ ]:
from datasets import load_from_disk
train_data = load_from_disk("artifacts/data/transformation/train")
train_data

In [ ]:
train_data.num_rows

In [ ]:
print(len(train_data[0]["input_ids"]), len(train_data[0]["attention_mask"]), len(train_data[0]["labels"]))

In [ ]:
import os
os.chdir("..")
os.getcwd()

In [ ]:
os.getcwd()

In [ ]:
from transformers import AutoTokenizer
def __transform(record:dict, tokenizer:AutoTokenizer, device:str) -> dict:
    # Pre-process input text
    tokenized_record_inputs =  tokenizer(record["dialogue"], truncation=True, max_length=512).to(device)
    with tokenizer.as_target_tokenizer():
        tokenized_record_outputs =  tokenizer(record["summary"], truncation=True, max_length=128).to(device)

    return {
        "input_ids": tokenized_record_inputs["input_ids"], 
        "attention_mask": tokenized_record_inputs["attention_mask"], 
        "labels": tokenized_record_outputs["input_ids"]
    }

In [ ]:
os.mkdir("less_records_artifacts")

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")
# model = AutoModelForSeq2SeqLM.from_pretrained("artifacts/model/base_estimator")
# Load a dataset from a CSV file
train_dataset = load_dataset('csv', data_files='notebook/data/train.csv')
validation_dataset = load_dataset('csv', data_files='notebook/data/validation.csv')
test_dataset = load_dataset('csv', data_files='notebook/data/test.csv')

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
train_data = train_dataset["train"].map(__transform, batched=True, fn_kwargs={"tokenizer":tokenizer, "device":device})
validation_data = validation_dataset["train"].map(__transform, batched=True, fn_kwargs={"tokenizer":tokenizer, "device":device})
test_data = test_dataset["train"].map(__transform, batched=True, fn_kwargs={"tokenizer":tokenizer, "device":device})

In [ ]:
train_data.save_to_disk("less_records_artifacts/train")
test_data.save_to_disk("less_records_artifacts/test")
validation_data.save_to_disk("less_records_artifacts/validation")

In [ ]:
import datasets
train_data = datasets.load_from_disk("less_records_artifacts/train")
validation_data = datasets.load_from_disk("less_records_artifacts/validation")
test_data = datasets.load_from_disk("less_records_artifacts/test")

In [ ]:
train_data

In [ ]:
from datasets import load_from_disk

test_dataset = load_from_disk("less_records_artifacts/test")
test_dataset

In [ ]:
[(feature, type(test_dataset[feature])) for feature in test_dataset.features]

In [ ]:
print([(feature, test_dataset[0][feature]) for feature in test_dataset.features])

In [ ]:
test_dataset["labels"]

In [ ]:
test_dataset["summary"]